<a href="https://colab.research.google.com/github/Takumi173/JPMA2023TF1-1/blob/main/JPMA2023_case1_(3)_%E3%83%A2%E3%83%87%E3%83%AB%E3%83%88%E3%83%AC%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前準備

## Google Driveの接続とパスの指定

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 拡張データの保存先ディレクトリの指定
datadir = '/content/drive/MyDrive/JPMA2023_case1/'

# 学習済みモデルの保存先ディレクトリの指定（ディレクトリは事前に用意する）
output_dir = '/content/drive/MyDrive/JPMA2023_case1/Models/'

Mounted at /content/drive


## ライブラリとパッケージのインストール

In [2]:
!pip install transformers[torch] fugashi[unidic-lite]

import pandas as pd
import torch
import statistics
import math
import os
from transformers import  AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers.trainer_utils import set_seed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidi

## データの読み込み

In [3]:
# Training data読み込み
Training_data = datadir + 'combined_training_data_watati_all.tsv'

df = pd.read_table(Training_data)
df

,ICD10_Term_WAKATI,Count,ICD10_CATEGORY,Label
0,いつも の コレラ 菌 による コレラ,1,A00,0
1,コレラ 詳細不明 の 悪化,1,A00,0
2,エルトールコレラ 菌 による コレラ から,1,A00,0
3,コレラ よる に 菌 エルトールコレラ,1,A00,0
4,不明 詳細 コレラ,1,A00,0
...,...,...,...,...
131396,鋭利 だ 物体 による 加害 に もとづく 傷害 及ぶ 死亡 工業 用 地域 及び 建築現場 で,1,X99,1525
131397,鋭利 だ 物体 による 加害 に もとづく 傷害 及ぶ 死亡 工業 用 地域 及び 建築現場...,1,X99,1525
131398,鋭利 だ 物体 による 加害 に もとづく 傷害 及ぶ 死亡 工業 用 地域 及び 建築現場...,1,X99,1525
131399,鋭利 だ 物体 による 加害 に もとづく 傷害 及ぶ 死亡 居住施設 から,1,X99,1525


# Trainingの準備

## Training dataを検証用に分割

In [4]:
# データをトレーニングセットとテストセットに分割（stratifyを使用）
docs = df["ICD10_Term_WAKATI"].tolist()
labels = df["Label"].tolist()

train_docs, test_docs, train_labels, test_labels =  train_test_split(docs, labels, test_size=0.2, random_state=42, stratify=labels)

# それぞれの件数確認
print('Training data  : ' + str(len(train_docs)))
print('Validation data: ' + str(len(test_docs)))

Training data  : 105120
Validation data: 26281


### Training dataの均衡化（適用する場合）

In [5]:
def data_equilibration(label, text, vol):
  data = {
      'Label': label,
      'ICD10_Term_WAKATI': text
      }

  df_ = pd.DataFrame(data).groupby('Label').sample(n=vol, replace=True, random_state=42)

  train_labels = df_['Label'].tolist()
  train_docs = df_['ICD10_Term_WAKATI'].tolist()
  return train_labels, train_docs

# 均一化を適用する場合は以下のコメントアウトを外して実行する
# train_labels, train_docs = data_equilibration(train_labels, train_docs, 60)

print('Training data  : ' + str(len(train_docs)))

Training data  : 105120


## データの確認

In [6]:
def count_labels(lst):
  sample_list = lst

  item_counts = Counter(sample_list)


  # 最大値（出現回数が最も多い項目）を取得
  max_item = max(item_counts, key=item_counts.get)
  max_count = item_counts[max_item]

  # 最小値（出現回数が最も少ない項目）を取得
  min_item = min(item_counts, key=item_counts.get)
  min_count = item_counts[min_item]

  print(f"最大値のID: {max_item} (出現回数: {max_count})")
  print(f"最小値のID: {min_item} (出現回数: {min_count})")
  return

print('*** train set ***')
count_labels(train_labels)
print('\n*** test set ***')
count_labels(test_labels)


## Code数のカウント
unique_count = len(set(labels))
print('\n*** 推論対象のコード数 ***')
print("ユニークカウント:", unique_count)

*** train set ***
最大値のID: 1462 (出現回数: 457)
最小値のID: 241 (出現回数: 2)

*** test set ***
最大値のID: 1462 (出現回数: 114)
最小値のID: 182 (出現回数: 1)

*** 推論対象のコード数 ***
ユニークカウント: 1526


## テキストの最大長確認

In [7]:
def max_text_length(text_list):
    if not text_list:  # 空リストの場合
        return 0

    max_length = max(len(text) for text in text_list)
    return max_length

print('***ALL***')
max_length = max_text_length(docs)
print(f"テキストの最大長は {max_length} 文字です。")

print('\n***train set***')
max_length = max_text_length(train_docs)
print(f"テキストの最大長は {max_length} 文字です。")

print('\n***test set***')
max_length = max_text_length(test_docs)
print(f"テキストの最大長は {max_length} 文字です。")


***ALL***
テキストの最大長は 131 文字です。

***train set***
テキストの最大長は 118 文字です。

***test set***
テキストの最大長は 131 文字です。


# Fine tuning

## モデル読み込み

In [8]:
# GPU 利用可の場合、GPUを利用
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

# 訓練時のシードを固定
set_seed(42)

In [9]:
model_name = "cl-tohoku/bert-base-japanese-v3"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=unique_count)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

for name, param in model.named_parameters():
    if 'bert.pooler.' in name:
        param.requires_grad = True
    elif 'classifier' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/447M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

In [10]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Tokenize

In [11]:
encodings = tokenizer(docs, return_tensors='pt',padding=True).to(device)
max_token_count = encodings.input_ids.size(1)
print("       最大トークン数:", max_token_count)

train_encodings = tokenizer(train_docs, return_tensors='pt', padding='max_length', max_length=max_token_count).to(device)
test_encodings = tokenizer(test_docs, return_tensors='pt', padding='max_length', max_length=max_token_count).to(device)

# トークン化された後の最大トークン数を取得
max_token_count_tr = train_encodings.input_ids.size(1)  # 入力テンソルのトークン数を取得
print("trainの最大トークン数:", max_token_count_tr)
max_token_count_ts = test_encodings.input_ids.size(1)  # 入力テンソルのトークン数を取得
print(" testの最大トークン数:", max_token_count_ts)

# Note: 最大トークンが大きくなりすぎるようであればtruncationとMax_lengthを手動設定する

       最大トークン数: 65
trainの最大トークン数: 65
 testの最大トークン数: 65


## 関数設定

In [12]:
# 精度計算関数の設定
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 学習の実行とモデルの保存

In [13]:
# ハイパーパラメータの設定
num_epoch = 16
batch_size = 64
lr_rate = 1e-3
num_warmup_steps  = math.ceil(len(train_docs) / batch_size) # 1 epochをウォームアップに使用

In [14]:
# Datasetを作成
class Prepare_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, index):
        item = {key: val[index].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[index])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Prepare_Dataset(train_encodings, train_labels)
test_dataset = Prepare_Dataset(test_encodings, test_labels)

In [15]:
training_args = TrainingArguments(
    output_dir="tmp",                       # モデルのアウトプットディレクトリ
    save_strategy='no',                     # チェックポイントの保存タイミング(no/epoch/steps)

    per_device_train_batch_size=batch_size, # トレーニングバッチサイズ
    per_device_eval_batch_size=2048,        # 検証バッチサイズ

    num_train_epochs=num_epoch,             # Training Epoch
    learning_rate= lr_rate,                 # ラーニングレート
    lr_scheduler_type="linear",             # 学習スケジューラ
    warmup_steps=num_warmup_steps,          # ウォームアップステップ

    logging_strategy ="epoch",              # ログの保存タイミング(no/epoch/steps)
    eval_strategy    ="epoch",              # 検証のタイミング(no/epoch/steps)

    dataloader_pin_memory = False,          # pin_memoryは無効化する
    )

trainer = Trainer(
    model = model,
    args  = training_args,
    train_dataset = train_dataset,
    eval_dataset  = test_dataset,
    compute_metrics = compute_metrics
    )

trainer.train()

# Note: 検証時のGPU RAM(T4)がギリギリなのでOut of memoryが出る場合はper_device_eval_batch_sizeの値を下げる。

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,4.875700,1.934159,0.562840,0.538552,0.642927,0.562840
2,1.620000,0.901216,0.776531,0.770038,0.819093,0.776531
3,1.115100,0.685392,0.828393,0.823872,0.856712,0.828393
4,0.932400,0.606268,0.850310,0.847972,0.876125,0.850310
5,0.827900,0.527594,0.870477,0.868331,0.891892,0.870477
6,0.745100,0.494785,0.883870,0.882886,0.902657,0.883870
7,0.683400,0.450211,0.895362,0.894928,0.911376,0.895362
8,0.629500,0.432986,0.900422,0.899436,0.914174,0.900422
9,0.580000,0.402621,0.908755,0.908214,0.919631,0.908755
10,0.536100,0.388406,0.915148,0.914511,0.924527,0.915148


TrainOutput(global_step=26288, training_loss=0.9413885440466538, metrics={'train_runtime': 7218.3962, 'train_samples_per_second': 233.005, 'train_steps_per_second': 3.642, 'total_flos': 5.69495312275968e+16, 'train_loss': 0.9413885440466538, 'epoch': 16.0})

In [16]:
# 訓練済みモデルの保存

n = len([d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]) # 既存のモデルが上書きされないようフォルダを分けて保存する

model.save_pretrained(output_dir + 'Model' + str(n))
tokenizer.save_pretrained (output_dir + 'Model' + str(n))

('/content/drive/MyDrive/JPMA2023_case1/Models/Model0/tokenizer_config.json',
 '/content/drive/MyDrive/JPMA2023_case1/Models/Model0/special_tokens_map.json',
 '/content/drive/MyDrive/JPMA2023_case1/Models/Model0/vocab.txt',
 '/content/drive/MyDrive/JPMA2023_case1/Models/Model0/added_tokens.json')